# 10分钟改变大模型自我认知，定制“专属自己”的聊天机器人

我们使用ms-swift对Qwen2.5-3B-Instruct进行自我认知微调。

- 模型：https://modelscope.cn/models/Qwen/Qwen2.5-3B-Instruct

- 自我认知数据集：https://modelscope.cn/datasets/swift/self-cognition

- 训练框架：https://github.com/modelscope/ms-swift.git

- 实验环境：A10、3090等（需显存资源12GB）

这里给出了两种训练和推理的方式，分别是：使用命令行界面和使用Python。

- 使用命令行界面：帮助开发者更快的将训练和推理跑起来。

- 使用Python：帮助开发者了解训练和推理的一些细节，这对定制训练过程有很大帮助。

准备好了吗？让我们开始这段旅程叭……

## 安装 ms-swift

In [8]:
%pip install ms-swift -U
%pip install transformers -U

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


## 使用Python


#### 训练
导入一些库：

In [ ]:
# 导入操作系统相关的库
import os
# 设置CUDA可见设备为第0块GPU
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

# 从swift.llm模块导入模型和分词器加载函数
# get_model_tokenizer: 获取模型和分词器
# load_dataset: 加载数据集
# get_template: 获取对话模板
# EncodePreprocessor: 文本编码预处理器
from swift.llm import get_model_tokenizer, load_dataset, get_template, EncodePreprocessor

# 从swift.utils模块导入工具函数
# get_logger: 获取日志记录器
# find_all_linears: 查找模型中所有线性层（用于LoRA）
# get_model_parameter_info: 获取模型参数信息
# plot_images: 绘制图像
# seed_everything: 设置随机种子确保实验可重复
from swift.utils import get_logger, find_all_linears, get_model_parameter_info, plot_images, seed_everything

# 从swift.tuners模块导入模型微调相关类
# Swift: 微调框架主类
# LoraConfig: LoRA（Low-Rank Adaptation）配置类
from swift.tuners import Swift, LoraConfig

# 从swift.trainers模块导入训练器和训练参数
# Seq2SeqTrainer: 序列到序列训练器
# Seq2SeqTrainingArguments: 序列到序列训练参数配置
from swift.trainers import Seq2SeqTrainer, Seq2SeqTrainingArguments

# 从functools导入partial函数，用于创建偏函数
from functools import partial

# 获取日志记录器实例
logger = get_logger()
# 设置全局随机种子为42，确保实验结果可重复
seed_everything(42)

[INFO:swift] Global seed set to 42


42

设置训练的超参数：

In [ ]:
# ========== 模型配置 ==========
# 指定要使用的模型路径或模型ID
model_id_or_path = '/home/work/hd/_models/base/qwen3-4b-thinking'  # model_id or model_path
# 设置系统提示语，定义AI助手的基本行为
system = 'You are a helpful assistant.'
# 设置训练输出目录，保存模型检查点和日志
output_dir = 'output'

# ========== 数据集配置 ==========
# 定义训练数据集列表，#后的数字表示使用该数据集的前N条数据
# AI-ModelScope/alpaca-gpt4-data-zh: 中文指令数据集，使用前500条
# AI-ModelScope/alpaca-gpt4-data-en: 英文指令数据集，使用前500条  
# swift/self-cognition: 自我认知数据集，使用前500条
dataset = ['AI-ModelScope/alpaca-gpt4-data-zh#500', 'AI-ModelScope/alpaca-gpt4-data-en#500',
           'swift/self-cognition#500']  # dataset_id or dataset_path
# 数据集随机种子，确保数据划分的可重复性
data_seed = 42
# 输入序列的最大长度（tokens数量）
max_length = 2048
# 验证集划分比例，0.01表示1%的数据用作验证集
split_dataset_ratio = 0.01  # 切分验证集
# 数据预处理时使用的进程数，加快数据处理速度
num_proc = 4  # 预处理的进程数

# ========== 自我认知数据集个性化配置 ==========
# 替换自我认知数据集中的填充符：{{NAME}}, {{AUTHOR}}
# 模型的中文名和英文名，用于替换数据集中的{{NAME}}占位符
model_name = ['精衡', 'Jingheng']  # 模型的中文名和英文名
# 模型作者的中文名和英文名，用于替换数据集中的{{AUTHOR}}占位符
model_author = ['叶博韬', 'YE SEVERIN']  # 模型作者的中文名和英文名

# ========== LoRA微调配置 ==========
# LoRA的秩（rank），控制低秩矩阵的维度，越小参数越少但表达能力较弱
lora_rank = 8
# LoRA的缩放因子，控制LoRA权重的影响程度，通常设为rank的4倍
lora_alpha = 32

# ========== 训练超参数配置 ==========
# 创建序列到序列训练参数对象
training_args = Seq2SeqTrainingArguments(
    # 模型检查点和日志的输出目录
    output_dir=output_dir,
    # 学习率，控制参数更新的步长
    learning_rate=1e-4,
    # 每个GPU上的训练批次大小
    per_device_train_batch_size=1,
    # 每个GPU上的验证批次大小
    per_device_eval_batch_size=1,
    # 启用梯度检查点，节省显存但增加计算时间
    gradient_checkpointing=True,
    # 权重衰减系数，防止过拟合的正则化技术
    weight_decay=0.1,
    # 学习率调度器类型，cosine表示余弦退火调度
    lr_scheduler_type='cosine',
    # 预热比例，训练初期逐渐增加学习率的步数占总步数的比例
    warmup_ratio=0.05,
    # 日志记录工具，tensorboard用于可视化训练过程
    report_to=['tensorboard'],
    # 是否在第一步就开始记录日志
    logging_first_step=True,
    # 模型保存策略，'steps'表示按步数保存
    save_strategy='steps',
    # 每隔50步保存一次模型检查点
    save_steps=50,
    # 模型评估策略，'steps'表示按步数评估
    eval_strategy='steps',
    # 每隔50步进行一次模型评估
    eval_steps=50,
    # 梯度累积步数，实际批次大小 = per_device_train_batch_size × gradient_accumulation_steps
    gradient_accumulation_steps=16,
    # 训练轮数，完整遍历数据集的次数
    num_train_epochs=1,
    # 选择最佳模型的评估指标，'loss'表示以损失函数为准
    metric_for_best_model='loss',
    # 最多保存的检查点数量，超过后删除最旧的
    save_total_limit=2,
    # 每隔5步记录一次训练日志
    logging_steps=5,
    # 数据加载器使用的工作进程数
    dataloader_num_workers=1,
    # 数据相关的随机种子
    data_seed=data_seed,
)

# 将输出目录转换为绝对路径
output_dir = os.path.abspath(os.path.expanduser(output_dir))
# 记录输出目录的绝对路径
logger.info(f'output_dir: {output_dir}')

[INFO:swift] output_dir: /home/work/hd/2_core/1_train/output


获取模型和对话template，并将可训练的lora层加入到模型中：

In [ ]:
# ========== 模型和分词器加载 ==========
# 加载指定路径的模型和分词器，指定模型类型为qwen3_thinking
# model_type参数用于区分同一目录下的不同模型变体
model, tokenizer = get_model_tokenizer(model_id_or_path, model_type='qwen3_thinking')
# 打印模型的基本信息（架构、参数量等）
logger.info(f'model_info: {model.model_info}')

# ========== 对话模板配置 ==========
# 获取模型对应的对话模板，用于格式化输入输出
# model.model_meta.template: 模型元数据中定义的模板类型
# default_system: 设置默认的系统提示语
# max_length: 限制输入序列的最大长度
template = get_template(model.model_meta.template, tokenizer, default_system=system, max_length=max_length)
# 将模板设置为训练模式，这会影响模板的行为（如是否包含特殊token）
template.set_mode('train')

# ========== LoRA配置和模型准备 ==========
# 查找模型中所有的线性层（Linear layers），这些层将被LoRA适配
# LoRA只对线性层进行低秩分解，不影响其他类型的层
target_modules = find_all_linears(model)

# 创建LoRA配置对象
lora_config = LoraConfig(
    task_type='CAUSAL_LM',  # 任务类型：因果语言模型（自回归生成）
    r=lora_rank,            # LoRA的秩，控制低秩矩阵的维度
    lora_alpha=lora_alpha,  # LoRA的缩放因子，控制适配权重的影响程度
    target_modules=target_modules  # 指定要应用LoRA的模块列表
)

# 使用Swift框架为模型添加LoRA适配器
# 这会在原模型的线性层旁边添加低秩分解的权重矩阵
model = Swift.prepare_model(model, lora_config)
# 打印LoRA配置信息
logger.info(f'lora_config: {lora_config}')

# ========== 模型信息输出 ==========
# 打印完整的模型结构（包括新添加的LoRA层）
logger.info(f'model: {model}')
# 获取并打印模型参数统计信息（总参数量、可训练参数量等）
model_parameter_info = get_model_parameter_info(model)
logger.info(f'model_parameter_info: {model_parameter_info}')

[INFO:swift] Setting torch_dtype: torch.bfloat16
[INFO:swift] model_kwargs: {'device_map': 'cuda:0'}
`torch_dtype` is deprecated! Use `dtype` instead!
[INFO:swift] model_kwargs: {'device_map': 'cuda:0'}
`torch_dtype` is deprecated! Use `dtype` instead!
Loading checkpoint shards: 100%|██████████| 3/3 [00:10<00:00,  3.63s/it]
[INFO:swift] model_info: ModelInfo(model_type='qwen3_thinking', model_dir='/home/work/hd/_models/base/qwen3-4b-thinking', torch_dtype=torch.bfloat16, max_model_len=262144, quant_method=None, quant_bits=None, rope_scaling=None, is_moe_model=False, config=Qwen3Config {
  "architectures": [
    "Qwen3ForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 151643,
  "dtype": "bfloat16",
  "eos_token_id": 151645,
  "head_dim": 128,
  "hidden_act": "silu",
  "hidden_size": 2560,
  "initializer_range": 0.02,
  "intermediate_size": 9728,
  "layer_types": [
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attentio

下载并载入数据集，并切分成训练集和验证集，

然后将文本编码成tokens：

In [ ]:
# ========== 数据集加载和划分 ==========
# 加载指定的数据集并自动划分为训练集和验证集
# dataset: 数据集列表，包含中文、英文指令数据和自我认知数据
# split_dataset_ratio: 验证集比例（0.01 = 1%）
# num_proc: 并行处理进程数，加快数据加载速度
# model_name/model_author: 用于替换自我认知数据集中的{{NAME}}和{{AUTHOR}}占位符
# seed: 随机种子，确保数据划分的可重复性
train_dataset, val_dataset = load_dataset(dataset, split_dataset_ratio=split_dataset_ratio, num_proc=num_proc,
        model_name=model_name, model_author=model_author, seed=data_seed)

# ========== 数据集信息输出 ==========
# 打印训练集的基本信息（数据条数、格式等）
logger.info(f'train_dataset: {train_dataset}')
# 打印验证集的基本信息
logger.info(f'val_dataset: {val_dataset}')
# 打印训练集第一条原始数据样本，查看数据格式和内容
logger.info(f'train_dataset[0]: {train_dataset[0]}')

# ========== 数据编码预处理 ==========
# 创建编码预处理器，将文本转换为模型可理解的token序列
# template: 使用之前配置的对话模板来格式化数据
# 对训练集进行编码处理：文本 -> token IDs
train_dataset = EncodePreprocessor(template=template)(train_dataset, num_proc=num_proc)
# 对验证集进行编码处理：文本 -> token IDs
val_dataset = EncodePreprocessor(template=template)(val_dataset, num_proc=num_proc)
# 打印编码后的训练集第一条数据，查看token化后的格式
logger.info(f'encoded_train_dataset[0]: {train_dataset[0]}')

# ========== 样本展示 ==========
# 使用模板的打印功能展示一条完整的训练样本
# 这会显示格式化后的输入输出对，包括系统提示、用户输入、模型回答等
template.print_inputs(train_dataset[0])

[INFO:swift] SelfCognitionPreprocessor has been successfully configured with name: ['精衡', 'Jingheng'], author: ['叶博韬', 'YE SEVERIN'].
[INFO:swift] Downloading the dataset from ModelScope, dataset_id: AI-ModelScope/alpaca-gpt4-data-zh
[INFO:swift] Downloading the dataset from ModelScope, dataset_id: AI-ModelScope/alpaca-gpt4-data-zh
[INFO:swift] Downloading the dataset from ModelScope, dataset_id: AI-ModelScope/alpaca-gpt4-data-en
[INFO:swift] Downloading the dataset from ModelScope, dataset_id: AI-ModelScope/alpaca-gpt4-data-en
[INFO:swift] Downloading the dataset from ModelScope, dataset_id: swift/self-cognition
[INFO:swift] Downloading the dataset from ModelScope, dataset_id: swift/self-cognition
Map (num_proc=4): 100%|██████████| 108/108 [00:00<00:00, 986.64 examples/s]
[WARNING:swift] dataset_sample:499 is greater than len(dataset):107, repeated sampling will be performed.

[WARNING:swift] dataset_sample:499 is greater than len(dataset):107, repeated sampling will be performed.
[IN

初始化trainer并开始训练：

In [14]:
model.enable_input_require_grads()  # 兼容gradient checkpointing
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    data_collator=template.data_collator,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    template=template,
)
trainer.train()

last_model_checkpoint = trainer.state.last_model_checkpoint
logger.info(f'last_model_checkpoint: {last_model_checkpoint}')

The model is already on multiple devices. Skipping the move to device specified in `args`.
/usr/bin/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
/usr/bin/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
/usr/bin/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
/usr/bin/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
[INFO:swift] use_reentrant: True
[INFO:swift] use_reentrant: True
The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None}.
The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokeniz

{'loss': 2.14067459, 'grad_norm': 2.05222344, 'learning_rate': 2e-05, 'token_acc': 0.55839002, 'epoch': 0.01, 'global_step/max_steps': '1/94', 'percentage': '1.06%', 'elapsed_time': '6s', 'remaining_time': '9m 48s', 'memory(GiB)': 8.91, 'train_speed(iter/s)': 0.157961}


Train:   5%|▌         | 5/94 [00:27<07:59,  5.39s/it]

{'loss': 1.93902969, 'grad_norm': 1.88177383, 'learning_rate': 0.0001, 'token_acc': 0.6216431, 'epoch': 0.05, 'global_step/max_steps': '5/94', 'percentage': '5.32%', 'elapsed_time': '27s', 'remaining_time': '8m 12s', 'memory(GiB)': 11.63, 'train_speed(iter/s)': 0.180765}


Train:  11%|█         | 10/94 [00:53<07:16,  5.20s/it]

...

{'loss': 1.11148405, 'grad_norm': 0.80076325, 'learning_rate': 5e-07, 'token_acc': 0.6948938, 'epoch': 0.97, 'global_step/max_steps': '90/94', 'percentage': '95.74%', 'elapsed_time': '7m 41s', 'remaining_time': '20s', 'memory(GiB)': 12.48, 'train_speed(iter/s)': 0.195222}


Val: 100%|██████████| 11/11 [00:00<00:00, 14.59it/s]


{'eval_loss': 1.2806505, 'eval_runtime': 0.8951, 'eval_samples_per_second': 12.289, 'eval_steps_per_second': 12.289, 'eval_token_acc': 0.62089552, 'epoch': 1.0, 'global_step/max_steps': '94/94', 'percentage': '100.00%', 'elapsed_time': '7m 57s', 'remaining_time': '0s', 'memory(GiB)': 12.89, 'train_speed(iter/s)': 0.196918}


[INFO:swift] Saving model checkpoint to /home/work/hd/2_core/1_train/output/checkpoint-94
Train: 100%|██████████| 94/94 [07:58<00:00,  5.09s/it]
[INFO:swift] last_model_checkpoint: /home/work/hd/2_core/1_train/output/checkpoint-94

[INFO:swift] last_model_checkpoint: /home/work/hd/2_core/1_train/output/checkpoint-94


{'train_runtime': 478.0244, 'train_samples_per_second': 3.115, 'train_steps_per_second': 0.197, 'train_loss': 1.29666271, 'epoch': 1.0, 'global_step/max_steps': '94/94', 'percentage': '100.00%', 'elapsed_time': '7m 58s', 'remaining_time': '0s', 'memory(GiB)': 12.89, 'train_speed(iter/s)': 0.196648}


可视化训练的loss。其中浅黄色线条代表真实loss值，黄色线条代表经过0.9平滑系数平滑后的loss值。

你也可以使用tensorboard进行实时可视化，在命令行输入`tensorboard --logdir '{output_dir}/runs'`。

In [ ]:
images_dir = os.path.join(output_dir, 'images')
logger.info(f'images_dir: {images_dir}')
plot_images(images_dir, training_args.logging_dir, ['train/loss'], 0.9)  # 保存图片

# 展示图片
from IPython.display import display
from PIL import Image
image = Image.open(os.path.join(images_dir, 'train_loss.png'))
display(image)

#### 微调后推理

导入一些库：

In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

from swift.llm import InferEngine, InferRequest, PtEngine, RequestConfig, get_template

设置推理的超参数：

In [2]:
last_model_checkpoint = 'output/vx-xxx/checkpoint-xxx'

# 模型
model_id_or_path = 'Qwen/Qwen2.5-3B-Instruct'  # model_id or model_path
system = 'You are a helpful assistant.'
infer_backend = 'pt'

# 生成参数
max_new_tokens = 512
temperature = 0
stream = True

获取推理引擎，并载入LoRA权重：

In [ ]:
engine = PtEngine(model_id_or_path, adapters=[last_model_checkpoint])
template = get_template(engine.model.model_meta.template, engine.tokenizer, default_system=system)
# 这里对推理引擎的默认template进行修改，也可以在`engine.infer`时进行传入
engine.default_template = template

开始推理...

In [ ]:
query_list = [
    'who are you?',
    "晚上睡不着觉怎么办？",
    '你是谁训练的？',
]

def infer_stream(engine: InferEngine, infer_request: InferRequest):
    request_config = RequestConfig(max_tokens=max_new_tokens, temperature=temperature, stream=True)
    gen_list = engine.infer([infer_request], request_config)
    query = infer_request.messages[0]['content']
    print(f'query: {query}\nresponse: ', end='')
    for resp in gen_list[0]:
        if resp is None:
            continue
        print(resp_list[0].choices[0].delta.content, end='', flush=True)
    print()

def infer(engine: InferEngine, infer_request: InferRequest):
    request_config = RequestConfig(max_tokens=max_new_tokens, temperature=temperature)
    resp_list = engine.infer([infer_request], request_config)
    query = infer_request.messages[0]['content']
    response = resp_list[0].choices[0].message.content
    print(f'query: {query}')
    print(f'response: {response}')

infer_func = infer_stream if stream else infer
for query in query_list:
    infer_func(engine, InferRequest(messages=[{'role': 'user', 'content': query}]))
    print('-' * 50)

## Web-UI

In [2]:
!CUDA_VISIBLE_DEVICES=0 \
swift web-ui \
    --adapters output/vx-xxx/checkpoint-xxx \
    --temperature 0 \
    --infer_backend pt \
    --max_new_tokens 2048

[WARNING:swift] Please use `swift app`.
run sh: `/home/work/hd/.venv/bin/python3 /home/work/.local/lib/python3.10/site-packages/swift/cli/app.py --adapters output/vx-xxx/checkpoint-xxx --temperature 0 --infer_backend pt --max_new_tokens 2048`
[INFO:swift] Successfully registered `/home/work/.local/lib/python3.10/site-packages/swift/llm/dataset/data/dataset_info.json`.
This can be used to load a bitsandbytes version built with a CUDA version that is different from the PyTorch CUDA version.
If this was unintended set the BNB_CUDA_VERSION variable to an empty string: export BNB_CUDA_VERSION=

[INFO:swift] Downloading the model from ModelScope Hub, model_id: output/vx-xxx/checkpoint-xxx
Traceback (most recent call last):
  File "/home/work/.local/lib/python3.10/site-packages/swift/cli/app.py", line 4, in <module>
    app_main()
  File "/home/work/.local/lib/python3.10/site-packages/swift/llm/app/app.py", line 44, in app_main
    return SwiftApp(args).main()
  File "/home/work/.local/lib/py